# 14.5 Takeaway - Build BST from Preorder and Postorder Traversal Data

Key thing: Order Matters, so inorder naturally will always be the same answer with a BST

Preorder and postorder is unique for each tree, so we can build an algorithm for each

## Data Structure

In [5]:
class BstNode:
    def __init__(self, data=None, left=None, right=None):
        self.data = data
        self.left = left
        self.right = right

    def printTree(self):
        printBTree(self,lambda n:(str(n.data),n.left,n.right))

import functools as fn

def printBTree(node, nodeInfo=None, inverted=False, isTop=True):

   # node value string and sub nodes
   stringValue, leftNode, rightNode = nodeInfo(node)

   stringValueWidth  = len(stringValue)

   # recurse to sub nodes to obtain line blocks on left and right
   leftTextBlock     = [] if not leftNode else printBTree(leftNode,nodeInfo,inverted,False)

   rightTextBlock    = [] if not rightNode else printBTree(rightNode,nodeInfo,inverted,False)

   # count common and maximum number of sub node lines
   commonLines       = min(len(leftTextBlock),len(rightTextBlock))
   subLevelLines     = max(len(rightTextBlock),len(leftTextBlock))

   # extend lines on shallower side to get same number of lines on both sides
   leftSubLines      = leftTextBlock  + [""] *  (subLevelLines - len(leftTextBlock))
   rightSubLines     = rightTextBlock + [""] *  (subLevelLines - len(rightTextBlock))

   # compute location of value or link bar for all left and right sub nodes
   #   * left node's value ends at line's width
   #   * right node's value starts after initial spaces
   leftLineWidths    = [ len(line) for line in leftSubLines  ]                            
   rightLineIndents  = [ len(line)-len(line.lstrip(" ")) for line in rightSubLines ]

   # top line value locations, will be used to determine position of current node & link bars
   firstLeftWidth    = (leftLineWidths   + [0])[0]  
   firstRightIndent  = (rightLineIndents + [0])[0] 

   # width of sub node link under node value (i.e. with slashes if any)
   # aims to center link bars under the value if value is wide enough
   # 
   # ValueLine:    v     vv    vvvvvv   vvvvv
   # LinkLine:    / \   /  \    /  \     / \ 
   #
   linkSpacing       = min(stringValueWidth, 2 - stringValueWidth % 2)
   leftLinkBar       = 1 if leftNode  else 0
   rightLinkBar      = 1 if rightNode else 0
   minLinkWidth      = leftLinkBar + linkSpacing + rightLinkBar
   valueOffset       = (stringValueWidth - linkSpacing) // 2

   # find optimal position for right side top node
   #   * must allow room for link bars above and between left and right top nodes
   #   * must not overlap lower level nodes on any given line (allow gap of minSpacing)
   #   * can be offset to the left if lower subNodes of right node 
   #     have no overlap with subNodes of left node                                                                                                                                 
   minSpacing        = 2
   rightNodePosition = fn.reduce(lambda r,i: max(r,i[0] + minSpacing + firstRightIndent - i[1]), \
                                 zip(leftLineWidths,rightLineIndents[0:commonLines]), \
                                 firstLeftWidth + minLinkWidth)

   # extend basic link bars (slashes) with underlines to reach left and right
   # top nodes.  
   #
   #        vvvvv
   #       __/ \__
   #      L       R
   #
   linkExtraWidth    = max(0, rightNodePosition - firstLeftWidth - minLinkWidth )
   rightLinkExtra    = linkExtraWidth // 2
   leftLinkExtra     = linkExtraWidth - rightLinkExtra

   # build value line taking into account left indent and link bar extension (on left side)
   valueIndent       = max(0, firstLeftWidth + leftLinkExtra + leftLinkBar - valueOffset)
   valueLine         = " " * max(0,valueIndent) + stringValue
   slash             = "\\" if inverted else  "/"
   backslash         = "/" if inverted else  "\\"
   uLine             = "¯" if inverted else  "_"

   # build left side of link line
   leftLink          = "" if not leftNode else ( " " * firstLeftWidth + uLine * leftLinkExtra + slash)

   # build right side of link line (includes blank spaces under top node value) 
   rightLinkOffset   = linkSpacing + valueOffset * (1 - leftLinkBar)                      
   rightLink         = "" if not rightNode else ( " " * rightLinkOffset + backslash + uLine * rightLinkExtra )

   # full link line (will be empty if there are no sub nodes)                                                                                                    
   linkLine          = leftLink + rightLink

   # will need to offset left side lines if right side sub nodes extend beyond left margin
   # can happen if left subtree is shorter (in height) than right side subtree                                                
   leftIndentWidth   = max(0,firstRightIndent - rightNodePosition) 
   leftIndent        = " " * leftIndentWidth
   indentedLeftLines = [ (leftIndent if line else "") + line for line in leftSubLines ]

   # compute distance between left and right sublines based on their value position
   # can be negative if leading spaces need to be removed from right side
   mergeOffsets      = [ len(line) for line in indentedLeftLines ]
   mergeOffsets      = [ leftIndentWidth + rightNodePosition - firstRightIndent - w for w in mergeOffsets ]
   mergeOffsets      = [ p if rightSubLines[i] else 0 for i,p in enumerate(mergeOffsets) ]

   # combine left and right lines using computed offsets
   #   * indented left sub lines
   #   * spaces between left and right lines
   #   * right sub line with extra leading blanks removed.
   mergedSubLines    = zip(range(len(mergeOffsets)), mergeOffsets, indentedLeftLines)
   mergedSubLines    = [ (i,p,line + (" " * max(0,p)) )       for i,p,line in mergedSubLines ]
   mergedSubLines    = [ line + rightSubLines[i][max(0,-p):]  for i,p,line in mergedSubLines ]                        

   # Assemble final result combining
   #  * node value string
   #  * link line (if any)
   #  * merged lines from left and right sub trees (if any)
   treeLines = [leftIndent + valueLine] + ( [] if not linkLine else [leftIndent + linkLine] ) + mergedSubLines

   # invert final result if requested
   treeLines = reversed(treeLines) if inverted and isTop else treeLines

   # return intermediate tree lines or print final result
   if isTop : print("\n".join(treeLines))
   else     : return treeLines      

## Idea 1

Preorder Idea 1: We can keep finding the root of each tree and subtree recursively

Example: [4, 2, 1, 3, 6, 5, 7]

- 4 is the root
- The transition point is where the first node is greater than the root (6)
  - root.left’s subtree = 2
    - pass in the subarray [2, 1, 3]
  - root.right’s subtree = 6
    - pass in the subarray [5, 7]

## Idea 1 Implementation

In [6]:
def rebuild_bst_from_preorder(preorder_sequence):
    if not preorder_sequence:
        return None
    root = BstNode(preorder_sequence[0])

    # Transition to the left by default
    transition_point = len(preorder_sequence)
    for i in range(len(preorder_sequence)):
        if preorder_sequence[i] > preorder_sequence[0]:
            transition_point = i
            break
    root.left = rebuild_bst_from_preorder(preorder_sequence[1:transition_point])
    root.right = rebuild_bst_from_preorder(preorder_sequence[transition_point:])
    return root

preorder_seq = [4, 2, 1, 3, 6, 5, 7]
built_tree = rebuild_bst_from_preorder(preorder_seq)


built_tree.printTree()

      4
   __/ \_
  2      6
 / \    / \
1   3  5   7


## Analysis

O(n^2) time complexity because we would have to iterate over nodes multiple times such as in a left-skewed tree. 
This would be O(nlogn) for a balanced tree.


## Idea 2

In the previous idea, the main issue is the for loop, which is causing us to reiterate over nodes when left skewed. 

Instead, we should build the nodes as we pass through the array. To account for when to stop a subtree, we should pass in a range of lower bound and upper bound, so that when that threshold is breached, we set the node to None, or in other words the subtree is complete.


## Implementation 2

In [7]:
def rebuild_bst_from_preorder_v2(preorder_sequence):

    def rebuild_bst_with_range(lower_bound, upper_bound):
        if root_idx[0] >= len(preorder_sequence):
            return None

        root = preorder_sequence[root_idx[0]]
        if not lower_bound <= root <= upper_bound:
            return None

        root_idx[0] += 1
        # Only natural to build the left subtree first in preorder sequence

        # nodes from here on out going left can’t be bigger than the current node
        left_tree = rebuild_bst_with_range(lower_bound, root)

        # nodes from here on out can’t be smaller than the current node
        right_tree = rebuild_bst_with_range(root, upper_bound)
        return BstNode(root, left_tree, right_tree)

    # Need to use an array instead of int b/c of python's weird reaction
    # to a global variable that's being assigned to (root_idx+=1) in next function scope
    root_idx = [0]
    return rebuild_bst_with_range(float('-inf'), float('inf'))


preorder_seq = [4, 2, 1, 3, 6, 5, 7]
built_tree = rebuild_bst_from_preorder_v2(preorder_seq)
built_tree.printTree()

      4
   __/ \_
  2      6
 / \    / \
1   3  5   7


## Weird Takeaway

We are using root_idx as an integer, but it's data structure is a list with a single integer in it. 

This is because it avoids this error:

```
variable referenced before assignment
```

When we assign `root_idx += 1`, this is assigning a value to a global variable, but the moment this assignment happens, this variable is local to the function and python freaks out.

Only when we do `root_idx[0] += 1` does python chill out about it as a global variable without having to be extra fancy about it..

## Analysis

This is O(n) since we go through each node only once at a time.



## Postorder Solution

This is very similar to the preorder solution.

Only difference is the way the traversal sequence is ordered. The root is at the very end of the array, and we visit the right tree, then left tree.

Easy way to handle this is to reverse the postorder_sequence, and flip the recursive tree visiting functions

In [8]:
def rebuild_bst_from_postorder(postorder_sequence):

    # Reverse the order for an easier time iterating through the list, since root is at the end
    postorder_sequence = postorder_sequence[::-1]
    def rebuild_bst_with_range(lower_bound, upper_bound):
        if root_idx[0] >= len(postorder_sequence):
            return None

        root = postorder_sequence[root_idx[0]]
        if not lower_bound <= root <= upper_bound:
            return None

        root_idx[0] += 1
        # It's natural to build the right subtree first in postorder sequence

        # nodes from here on out can’t be smaller than the current node
        right_tree = rebuild_bst_with_range(root, upper_bound)

        # nodes from here on out going left can’t be bigger than the current node
        left_tree = rebuild_bst_with_range(lower_bound, root)


        return BstNode(root, left_tree, right_tree)

    # Need to use an array instead of int b/c of python's weird reaction
    # to a global variable that's being assigned to (root_idx+=1) in next function scope
    root_idx = [0]
    return rebuild_bst_with_range(float('-inf'), float('inf'))


postorder_seq = [1, 3, 2, 5, 7, 6, 4]
built_tree = rebuild_bst_from_postorder(postorder_seq)
built_tree.printTree()

      4
   __/ \_
  2      6
 / \    / \
1   3  5   7
